# Pre Requesits

In [ ]:
# pip install tensorflow
# pip install keras
# pip install numpy
# pip install sklearn
# pip install opencv-python
# pip install scikit-learn
# pip install tensorflow-gpu

# Libraries

In [ ]:
import pandas as pd
import numpy as np
import warnings

# Model Training
import sys, os
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.callbacks import ReduceLROnPlateau, TensorBoard, EarlyStopping, ModelCheckpoint
from keras.models import load_model
from keras.models import model_from_json

# Model Testing
from __future__ import division
from keras.models import Sequential
from keras.layers import Dense
from keras.models import model_from_json

# Confusion Matrix
import itertools
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

# FER
import cv2

# Constants and Variables

In [ ]:
PATH = '/home/nicholaspereira/Downloads/Facial Emotion Recognition/'

# Main

## Pre Processing

In [ ]:
warnings.filterwarnings("ignore")
data = pd.read_csv(f'{PATH}fer2013.csv')
width, height = 48, 48
datapoints = data['pixels'].tolist()

#getting features for training
X = []
for xseq in datapoints:
    xx = [int(xp) for xp in xseq.split(' ')]
    xx = np.asarray(xx).reshape(width, height)
    X.append(xx.astype('float32'))

X = np.asarray(X)
X = np.expand_dims(X, -1)

#getting labels for training
y = pd.get_dummies(data['emotion']).values
#storing them using numpy
np.save(f'{PATH}fdataX', X)
np.save(f'{PATH}flabels', y)

print("Preprocessing Done")
print("Number of Features: " + str(len(X[0])))
print("Number of Labels: " + str(len(y[0])))
print("Number of examples in dataset:" + str(len(X)))
print("X,y stored in fdataX.npy and flabels.npy respectively")

## Model Training

In [ ]:
num_features = 64
num_labels = 7
batch_size = 64
epochs = 100
width, height = 48, 48

x = np.load(f'{PATH}fdataX.npy')
y = np.load(f'{PATH}flabels.npy')

x -= np.mean(x, axis=0)
x /= np.std(x, axis=0)

#for xx in range(10):
#    plt.figure(xx)
#    plt.imshow(x[xx].reshape((48, 48)), interpolation='none', cmap='gray')
#plt.show()

#splitting into training, validation and testing data
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1, random_state=41)

#saving the test samples to be used later
np.save(f'{PATH}modXtest', X_test)
np.save(f'{PATH}modytest', y_test)

#desinging the CNN
model = Sequential()

model.add(Conv2D(num_features, kernel_size=(3, 3), activation='relu', input_shape=(width, height, 1), data_format='channels_last', kernel_regularizer=l2(0.01)))
model.add(Conv2D(num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(2*2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(2*2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(2*2*2*num_features, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(2*2*num_features, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(2*num_features, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(num_labels, activation='softmax'))

#model.summary()

#Compliling the model with adam optimixer and categorical crossentropy loss
model.compile(loss=categorical_crossentropy,
              optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-7),
              metrics=['accuracy'])

#training the model
model.fit(np.array(X_train), np.array(y_train),
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(np.array(X_valid), np.array(y_valid)),
          shuffle=True)

#saving the model to be used later
fer_json = model.to_json()
with open(f'{PATH}fer.json', 'w') as json_file:
    json_file.write(fer_json)
model.save_weights(f'{PATH}fer.h5')
print("Saved model to disk")

## Running Some Tests

In [ ]:
json_file = open(f'{PATH}fer.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights(f'{PATH}fer.h5')
print("Loaded model from disk")

truey=[]
predy=[]
x = np.load(f'{PATH}modXtest.npy')
y = np.load(f'{PATH}modytest.npy')

yhat= loaded_model.predict(x)
yh = yhat.tolist()
yt = y.tolist()
count = 0

for i in range(len(y)):
    yy = max(yh[i])
    yyt = max(yt[i])
    predy.append(yh[i].index(yy))
    truey.append(yt[i].index(yyt))
    if(yh[i].index(yy)== yt[i].index(yyt)):
        count+=1

acc = (count/len(y))*100

#saving values for confusion matrix and analysis
np.save(f'{PATH}truey', truey)
np.save(f'{PATH}predy', predy)
print("Predicted and true label values saved")
print("Accuracy on test set: " + str(round(acc,2)) + "%")

## Confusion Matrix

In [ ]:
y_true = np.load(f'{PATH}truey.npy')
y_pred = np.load(f'{PATH}predy.npy')
cm = confusion_matrix(y_true, y_pred)
labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
title = 'Confusion matrix'
print(cm)

plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title(title)
plt.colorbar()
tick_marks = np.arange(len(labels))
plt.xticks(tick_marks, labels, rotation=45)
plt.yticks(tick_marks, labels)
fmt = 'd'
thresh = cm.max() / 2.
for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    plt.text(j, i, format(cm[i, j], fmt),
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")

plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.tight_layout()
plt.show()

# FER

## Image Facial Emotion Recognition Test

In [ ]:
# # Loading the model
# json_file = open(f'{PATH}fer.json', 'r')
# loaded_model_json = json_file.read()
# json_file.close()
# loaded_model = model_from_json(loaded_model_json)
# # Loading weights into new model
# loaded_model.load_weights(f'{PATH}fer.h5')
# print("Loaded model from disk")

# # Setting image resizing parameters
# WIDTH = 48
# HEIGHT = 48
# x = None
# y = None

# # Loading image
# full_size_image = cv2.imread(f'{PATH}test.jpg')
# print("Image Loaded")
# gray=cv2.cvtColor(full_size_image,cv2.COLOR_RGB2GRAY)
# face = cv2.CascadeClassifier(f'{PATH}haarcascade_frontalface_default.xml')
# faces = face.detectMultiScale(gray, 1.3  , 10)

# # Detecting faces
# for (x, y, w, h) in faces:
#         roi_gray = gray[y:y + h, x:x + w]
#         cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray, (48, 48)), -1), 0)
#         cv2.normalize(cropped_img, cropped_img, alpha=0, beta=1, norm_type=cv2.NORM_L2, dtype=cv2.CV_32F)
#         cv2.rectangle(full_size_image, (x, y), (x + w, y + h), (255, 0, 0), 1)
#         # Predicting the emotion
#         yhat= loaded_model.predict(cropped_img)
#         cv2.putText(full_size_image, labels[int(np.argmax(yhat))], (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 1, cv2.LINE_AA)
#         print("Emotion: " + labels[int(np.argmax(yhat))])

# cv2.imshow('Emotion', full_size_image)
# cv2.waitKey()
# cv2.destroyAllWindows()

## Live Facial Emotion Recognition

In [ ]:
camera = cv2.VideoCapture(0)
face_cascade = cv2.CascadeClassifier(f'{PATH}haarcascade_frontalface_default.xml')

# Inicializar DataFrame vazio
df_sentimentos = pd.DataFrame(columns=['Tempo', 'Sentimento'])

# Starting counter and current sentiment
count = 0
current_sentiment = None

while True:
    ret, frame = camera.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        face_img = gray[y:y + h, x:x + w]
        resized_img = cv2.resize(face_img, (48, 48))
        normalized_img = resized_img / 255.0
        reshaped_img = np.reshape(normalized_img, (1, 48, 48, 1))

        # Predicting sentiments
        emotion = labels[np.argmax(loaded_model.predict(reshaped_img))]
        
        # Drawing rectangle around the face and showing predicted sentiment
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
        cv2.putText(frame, emotion, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

        # Adicionar sentimento ao DataFrame
        df_sentimentos = df_sentimentos.append({'Tempo': pd.Timestamp.now(), 'Sentimento': emotion}, ignore_index=True)
    
    # Predicting sentiments
    previsoes = loaded_model.predict(reshaped_img)
    novo_sentimento = labels[np.argmax(previsoes)]

    # Check if current sentiment is the same as the latest sentiment 
    if current_sentiment is not None and current_sentiment == novo_sentimento:
        count += 1
    else:
        count = 1
        current_sentiment = novo_sentimento
    
    # Check if it predicted the same sentiments three times in a row
    if count >= 3:
        # Show sentiment on the window
        cv2.imshow('Sentiment Detection', frame)

        # Print sentiment on console
        print("Sentimento:", current_sentiment)

        # Verificar se já se passaram 15 segundos desde a última impressão no DataFrame
        ultimo_registro = df_sentimentos.iloc[-1] if not df_sentimentos.empty else None
        if ultimo_registro is None or pd.Timestamp.now() - ultimo_registro['Tempo'] >= pd.Timedelta(seconds=15):
            # Imprimir DataFrame com os sentimentos
            print(df_sentimentos)
            print()

    # End loop if 'q' key was pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Clear camera and close windows
camera.release()
cv2.destroyAllWindows()

# Calcular a contagem de cada sentimento
contagem_sentimentos = df_sentimentos['Sentimento'].value_counts()

# Calcular a porcentagem de cada sentimento
porcentagem_sentimentos = contagem_sentimentos / len(df_sentimentos) * 100

# Adicionar a coluna de porcentagem ao DataFrame
df_sentimentos['Porcentagem'] = porcentagem_sentimentos

df_sentimentos.to_csv(f'{PATH}Detecção de Sentimentos em Tempo Real - 19-06-23.csv', index = False)